<a href="https://colab.research.google.com/github/deacs11/CrewAI_Podcast_Episode_Planner_-_Script_Outline/blob/main/CrewAI_Podcast_Episode_Planner_%26_Script_Outline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# @title 1. Install necessary libraries
# Installs crewai, tools, OpenAI client, Colab module, and search tool
!pip install crewai crewai-tools langchain-openai google-colab duckduckgo-search==5.3.1b1 -q

print("Library installation completed!")

Library installation completed!


In [3]:
# @title 2. Import modules and configure API Keys from Secrets
import os
from google.colab import userdata # To read Colab secrets
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, WebsiteSearchTool # Tools for research
from langchain_openai import ChatOpenAI

# --- API KEY CONFIGURATION FROM COLAB SECRETS ---
print("Attempting to load API Keys from Secrets...")
try:
    # Reads the OpenAI key from the secret 'OPENAI_API_KEY'
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    # Reads the Serper key from the secret 'SERPER_API_KEY'
    os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')
    print("-> API Key environment variables set successfully.")
except Exception as e:
    print(f"!!! Error loading API Keys from Secrets: {e}")
    print("!!! Make sure you have set them correctly in the Colab Secrets panel")
    print("!!! and enabled 'Notebook access' for both.")

# Debug block to verify keys
print("-" * 20)
retrieved_key = os.environ.get("OPENAI_API_KEY")
if retrieved_key:
    print(f"OpenAI Key FOUND in environment: '{retrieved_key[:5]}...{retrieved_key[-4:]}'")
else:
    print("!!! OpenAI Key NOT FOUND.")
print("-" * 20)
retrieved_serper_key = os.environ.get("SERPER_API_KEY")
if retrieved_serper_key:
     print(f"Serper Key FOUND in environment: '{retrieved_serper_key[:5]}...{retrieved_serper_key[-4:]}'")
else:
    print("--- Serper Key NOT FOUND (required for TopicResearcher Agent).")
print("-" * 20)

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "schema" in "DatabricksQueryToolSchema" shadows an attribute in parent "BaseModel"
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:617: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/usr/local/lib/python3.11/dist-packages/crewai_t

Attempting to load API Keys from Secrets...
-> API Key environment variables set successfully.
--------------------
OpenAI Key FOUND in environment: 'sk-pr...DKkA'
--------------------
Serper Key FOUND in environment: '7354c...dc56'
--------------------


In [4]:
# @title 3. Define podcast episode details

# --- MODIFY THESE VARIABLES FOR YOUR EPISODE ---
episode_topic = "The future of remote work and its impact on company culture"
# Optional Guest Info (set guest_name to None or "" if no guest)
guest_name = "Dr. Aria Patel"
guest_info_url = "https://example.com/experts/aria-patel-profile" # e.g., LinkedIn, personal site, bio page
# Optional Style Info
podcast_style = "Interview format, conversational but informative tone. Target audience: HR professionals and managers. Approx 30-40 mins."
# ---------------------------------------------------------------------

print("Podcast episode details set:")
print(f"- Topic: {episode_topic}")
if guest_name:
    print(f"- Guest: {guest_name}")
    print(f"- Guest info URL: {guest_info_url}")
else:
    print("- Format: Solo episode (no guest specified)")
if podcast_style:
    print(f"- Style Notes: {podcast_style}")

# Store guest presence for easy checking later
has_guest = bool(guest_name and guest_name.strip())

Podcast episode details set:
- Topic: The future of remote work and its impact on company culture
- Guest: Dr. Aria Patel
- Guest info URL: https://example.com/experts/aria-patel-profile
- Style Notes: Interview format, conversational but informative tone. Target audience: HR professionals and managers. Approx 30-40 mins.


In [5]:
# @title 4. Select LLM and initialize Tools

# --- CHOOSE THE LANGUAGE MODEL (LLM) ---
try:
    # GPT-4 recommended for better structure, creativity, and research synthesis
    llm = ChatOpenAI(model="gpt-4-turbo")
    # llm = ChatOpenAI(model="gpt-3.5-turbo") # Cheaper, might be less structured
    print(f"LLM ({llm.model_name}) initialized successfully.")
except Exception as e:
    print(f"!!! Error initializing ChatOpenAI: {e}")
    print("!!! Verify OpenAI API key.")
    raise

# --- INITIALIZE TOOLS ---
# Tools needed for the TopicResearcher
search_tool = None
web_reader_tool = None
try:
    search_tool = SerperDevTool()
    print("SerperDevTool initialized.")
    web_reader_tool = WebsiteSearchTool()
    print("WebsiteSearchTool initialized.")
except Exception as e:
    print(f"--- Warning: Error initializing tools: {e}. Research capabilities might be limited.")

available_tools = [tool for tool in [search_tool, web_reader_tool] if tool is not None]
print(f"Tools actually available for research: {[t.name for t in available_tools] if available_tools else 'None'}")

LLM (gpt-4-turbo) initialized successfully.
SerperDevTool initialized.
WebsiteSearchTool initialized.
Tools actually available for research: ['Search the internet with Serper', 'Search in a specific website']


/usr/local/lib/python3.11/dist-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing this attribute on the instance is deprecated, and will be removed in Pydantic V3. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x


In [7]:
# @title 5. Define Agents for Podcast Planning Crew

print("Defining Podcast Planning Agents...")

if 'llm' not in locals() or llm is None:
     raise ValueError("LLM not initialized.")
if not available_tools:
     print("--- Warning: No research tools available. TopicResearcher agent will be limited.")

# --- Agent 1: Topic Researcher ---
topic_researcher = Agent(
    role='Podcast Research Specialist',
    goal=(
        f"Conduct thorough research on the episode topic: '{episode_topic}'. "
        + (f"Additionally, research the guest '{guest_name}' using their provided info URL ({guest_info_url}) and general web search. "
           f"Focus on their expertise, recent work, notable opinions related to '{episode_topic}', and any interesting personal anecdotes or projects." if has_guest else "")
        + "Synthesize findings into key facts, interesting angles, potential discussion points, and relevant statistics."
    ),
    backstory=(
        "You are an expert researcher skilled at digging deep into topics and people. You know how to use search engines and read websites "
        "to find compelling information, separating signal from noise. You aim to provide the podcast host with rich, accurate material "
        "to make the episode insightful and engaging, whether it's a solo commentary or an interview."
    ),
    tools=available_tools, # Requires search and web reading tools
    llm=llm,
    allow_delegation=False,
    verbose=True,
    max_iter=7 # Allow more iterations for potentially researching topic + guest
)
print("- Agent 'topic_researcher' defined.")

# --- Agent 2: Episode Structurer ---
episode_structurer = Agent(
    role='Podcast Producer / Show Flow Designer',
    goal=(
        f"Develop a logical and engaging segment structure for a podcast episode on '{episode_topic}' "
        + (f"featuring guest '{guest_name}'. " if has_guest else ". ")
        + f"Consider the style: '{podcast_style}'. Propose 4-6 distinct segments (e.g., Intro, Topic Deep Dive, Guest Insights, Listener Questions (placeholder), Key Takeaways, Outro) "
        f"with estimated durations for each, summing to roughly 30-40 minutes."
    ),
    backstory=(
        "You are a seasoned podcast producer who understands how to create a compelling narrative arc for listeners. "
        "You structure episodes logically, ensuring a smooth flow between topics, managing time effectively, and building towards a satisfying conclusion. "
        "You adapt the structure based on whether it's an interview or solo format."
    ),
    tools=[],
    llm=llm,
    allow_delegation=False,
    verbose=True,
    max_iter=3
)
print("- Agent 'episode_structurer' defined.")

# --- Agent 3: Content Point Generator ---
content_point_generator = Agent(
    role='Podcast Content & Question Writer',
    goal=(
        "Based on the research findings (topic context & guest info if applicable) and the proposed episode structure, generate specific, engaging talking points, key facts, or discussion prompts for *each* segment. "
        + (f"If a guest ('{guest_name}') is present, draft 3-5 insightful and open-ended questions specifically tailored to their expertise and the segment's focus. " if has_guest else "")
        + "Ensure points/questions align with the segment's purpose within the overall structure."
    ),
    backstory=(
        "You excel at transforming research into compelling podcast content. You craft talking points that spark interest and generate "
        "thought-provoking questions for guests. You ensure that the content for each segment directly supports the episode's goals and structure."
    ),
    tools=[], # Works off context from previous tasks
    llm=llm,
    allow_delegation=False,
    verbose=True,
    max_iter=5
)
print("- Agent 'content_point_generator' defined.")

# --- Agent 4: Script Element Writer ---
script_element_writer = Agent(
    role='Podcast Script Snippet Writer',
    goal=(
        f"Draft concise text snippets for the episode's key scripted elements: \n"
        f"1. Introduction: Hook for '{episode_topic}', brief topic intro" + (f", and engaging introduction for the guest '{guest_name}'." if has_guest else ".") + "\n"
        f"2. Outro: Summary of 1-2 key takeaways, a clear call to action (e.g., subscribe, visit website), and thank you" + (f" to the guest '{guest_name}'." if has_guest else ".")
    ),
    backstory=(
        "You specialize in writing impactful beginnings and endings. You know how to craft hooks that grab attention, introduce topics and guests smoothly, "
        "summarize value effectively, and create clear calls to action. Your snippets provide the host with polished language for crucial parts of the show."
    ),
    tools=[],
    llm=llm,
    allow_delegation=False,
    verbose=True,
    max_iter=3
)
print("- Agent 'script_element_writer' defined.")

# --- Agent 5: Plan Compiler ---
plan_compiler = Agent(
    role='Episode Plan Consolidator',
    goal='Compile all the generated elements (research summary, episode structure with timings, talking points/questions per segment, intro/outro snippets) into a single, well-organized, and easy-to-read podcast episode plan document using Markdown.',
    backstory=(
        "You are the ultimate organizer. You take inputs from various specialists and assemble them into a practical, unified plan that a podcast host "
        "can easily follow during recording. Clarity, structure, and completeness are your priorities. You ensure all necessary information is included logically."
    ),
    tools=[],
    llm=llm,
    allow_delegation=False,
    verbose=True,
    max_iter=3
)
print("- Agent 'plan_compiler' defined.")

print("All podcast planning agents defined.")

Defining Podcast Planning Agents...
- Agent 'topic_researcher' defined.
- Agent 'episode_structurer' defined.
- Agent 'content_point_generator' defined.
- Agent 'script_element_writer' defined.
- Agent 'plan_compiler' defined.
All podcast planning agents defined.


In [8]:
# @title 6. Define Tasks for Podcast Planning Crew

print("Defining Podcast Planning Tasks...")

# Verify agents are defined
if 'topic_researcher' not in locals() or \
   'episode_structurer' not in locals() or \
   'content_point_generator' not in locals() or \
   'script_element_writer' not in locals() or \
   'plan_compiler' not in locals():
    raise ValueError("One or more required agents are not defined.")

# --- Task 1: Research topic and guest ---
task_research = Task(
    description=(
        f"Conduct research based on the episode topic '{episode_topic}'"
        + (f", the guest '{guest_name}', and their info URL '{guest_info_url}'. " if has_guest else ". ")
        + f"Use the provided web search and reading tools. Focus on finding key facts, interesting angles, statistics, "
        + ("and the guest's relevant expertise, recent work, and opinions on the topic. " if has_guest else "")
        + "Synthesize the findings into a concise research summary."
    ),
    agent=topic_researcher,
    expected_output=(
        "A research summary document containing key findings about the topic "
        + ("and the guest (if applicable). Include bullet points for easy scanning of facts, angles, and guest background/expertise." if has_guest else ". Include bullet points for easy scanning.")
    )
)
print("- Task 'task_research' defined.")

# --- Task 2: Structure the episode ---
task_structure = Task(
    description=(
        f"Based on the episode topic '{episode_topic}'"
        + (f", the guest '{guest_name}', " if has_guest else ", ")
        + f"and the style notes '{podcast_style}', create a logical segment structure for the episode. "
        f"Aim for 4-6 segments with brief descriptive titles and estimated timings (adding up to ~30-40 mins). "
        f"Present as a numbered list or outline."
    ),
    agent=episode_structurer,
    expected_output=(
        "A structured outline of the podcast episode segments (4-6 items). Each item should include: "
        "Segment number, descriptive title, and estimated duration (e.g., 'Segment 1: Intro & Hook (3 mins)')."
    ),
    context=[task_research] # Structure can be informed by the depth of research findings
)
print("- Task 'task_structure' defined.")

# --- Task 3: Generate content points & questions ---
task_content_points = Task(
    description=(
        "Develop detailed content for the episode based on the research summary and the segment structure (both provided in context). "
        "For each segment defined in the structure, generate specific talking points, key facts, anecdotes, or discussion prompts. "
        + (f"Crucially, for segments involving the guest '{guest_name}', draft 3-5 open-ended, insightful questions tailored to their expertise as identified in the research." if has_guest else "")
        + "Organize the output clearly, associating points/questions with their respective segments from the structure."
    ),
    agent=content_point_generator,
    expected_output=(
        "Detailed content outline mapping talking points and specific questions (if guest) to each segment defined in the episode structure. "
        "Example:\nSegment 2: Defining Remote Culture (10 mins)\n- Talking Point: Discuss common misconceptions...\n- Fact: Statistic about remote work growth...\n"
        + ("- Question for Guest: Dr. Patel, how do you see..." if has_guest else "")
    ),
    context=[task_research, task_structure] # Needs both research and structure
)
print("- Task 'task_content_points' defined.")

# --- Task 4: Draft script elements ---
task_script_elements = Task(
    description=(
        f"Write draft text snippets for the host's Introduction and Outro, based on the episode topic '{episode_topic}' and guest '{guest_name}' (if applicable, info in research context). "
        f"The Intro should include an engaging hook, topic setup, " + ("and guest introduction. " if has_guest else ". ") +
        f"The Outro should summarize 1-2 key takeaways, include a call to action, and thank " + ("the guest." if has_guest else "the listeners.")
    ),
    agent=script_element_writer,
    expected_output=(
        "Two distinct text blocks: one labeled 'Draft Introduction Snippet' and one labeled 'Draft Outro Snippet', containing the drafted text for the host."
    ),
    context=[task_research] # Needs research mainly for topic/guest details
)
print("- Task 'task_script_elements' defined.")

# --- Task 5: Compile final episode plan ---
task_compile_plan = Task(
    description=(
        "Assemble all the previously generated components (research summary, episode structure, content points/questions per segment, intro/outro snippets - all provided in context) "
        "into a single, comprehensive, and well-organized podcast episode plan document. "
        "Use Markdown formatting (headings, lists) for clarity. Ensure all parts are included logically under appropriate headings."
    ),
    agent=plan_compiler,
    expected_output=(
        "A complete podcast episode plan document in Markdown format. It should contain clearly separated sections for: "
        "1. Episode Overview (Topic, Guest if any) "
        "2. Research Summary Highlights "
        "3. Episode Structure (Segments with Timings) "
        "4. Detailed Segment Breakdown (Talking Points & Questions per segment) "
        "5. Draft Intro Snippet "
        "6. Draft Outro Snippet. "
        "The document should be ready for a host to use."
    ),
    # Needs all generated pieces to compile the final plan
    context=[task_research, task_structure, task_content_points, task_script_elements]
)
print("- Task 'task_compile_plan' defined.")

print("All podcast planning tasks defined.")

Defining Podcast Planning Tasks...
- Task 'task_research' defined.
- Task 'task_structure' defined.
- Task 'task_content_points' defined.
- Task 'task_script_elements' defined.
- Task 'task_compile_plan' defined.
All podcast planning tasks defined.


In [9]:
# @title 7. Create and run podcast Planning Crew

print("Creating the Podcast Planning Crew...")

# Verify all components are ready
if 'topic_researcher' not in locals() or \
   'episode_structurer' not in locals() or \
   'content_point_generator' not in locals() or \
   'script_element_writer' not in locals() or \
   'plan_compiler' not in locals() or \
   'task_research' not in locals() or \
   'task_structure' not in locals() or \
   'task_content_points' not in locals() or \
   'task_script_elements' not in locals() or \
   'task_compile_plan' not in locals():
    raise ValueError("Missing agents or tasks needed to create the Podcast Planning Crew.")

# Assemble the Crew
podcast_planning_crew = Crew(
    agents=[topic_researcher, episode_structurer, content_point_generator, script_element_writer, plan_compiler],
    tasks=[task_research, task_structure, task_content_points, task_script_elements, task_compile_plan],
    process=Process.sequential,
    memory=True,
    cache=True,
    verbose=True
)

print("Podcast Planning Crew created. Starting the process with kickoff()...")

# Define initial inputs
crew_inputs = {
    'episode_topic': episode_topic,
    'guest_name': guest_name,
    'guest_info_url': guest_info_url,
    'podcast_style': podcast_style
}

# Kick off the crew's work
result = podcast_planning_crew.kickoff(inputs=crew_inputs)

print("\n\n*****************************************")
print("   PODCAST PLANNING CREW FINISHED WORK!   ")
print("*****************************************")

Creating the Podcast Planning Crew...


/usr/local/lib/python3.11/dist-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing this attribute on the instance is deprecated, and will be removed in Pydantic V3. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x


Podcast Planning Crew created. Starting the process with kickoff()...


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: f4a0c33a-87a2-4047-98c5-8840131bd647                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 8c5dc686-2fa2-4e70-ab62-ff0695db31ff
       Status: Executing Task...

...

🚀 Crew: crew
├── 📋 Task: 8c5dc686-2fa2-4e70-ab62-ff0695db31ff
│      Assigned to: Podcast Research Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Podcast Research Specialist
│           Status: ✅ Completed
│       ├── 🔧 Failed Search in a specific website (1)
│       └── 🔧 Used Search the internet with Serper (3)
├── 📋 Task: 9817858c-a95a-4fec-ba55-29c85bd21dc4
│      Assigned to: Podcast Producer / Show Flow Designer
│      Status: ✅ Completed
│   └── 🤖 Agent: Podcast Producer / Show Flow Designer
│           Status: ✅ Completed
├── 📋 Task: 7921702c-1926-441a-af4f-b462926bfeba
│      Assigned to: Podcast Content & Question Writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Podcast Content & Question Writer
│           Status: ✅ Completed
├── 📋 Task: 77820b02-7690-4969-a89f-ac1af1c80435
│      Assigned to: Podcast Script Snippet Writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Podcast Script Snippet Writer
│           Status: ✅ Completed
└── 📋 Task: 2d38dbbd-6633-4b99-a50d-8be98c1390df
       Status: Executing Task...
    └── 🤖 Agent: Episode Plan Consolidator
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 8c5dc686-2fa2-4e70-ab62-ff0695db31ff
│      Assigned to: Podcast Research Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Podcast Research Specialist
│           Status: ✅ Completed
│       ├── 🔧 Failed Search in a specific website (1)
│       └── 🔧 Used Search the internet with Serper (3)
├── 📋 Task: 9817858c-a95a-4fec-ba55-29c85bd21dc4
│      Assigned to: Podcast Producer / Show Flow Designer
│      Status: ✅ Completed
│   └── 🤖 Agent: Podcast Producer / Show Flow Designer
│           Status: ✅ Completed
├── 📋 Task: 7921702c-1926-441a-af4f-b462926bfeba
│      Assigned to: Podcast Content & Question Writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Podcast Content & Question Writer
│           Status: ✅ Completed
├── 📋 Task: 77820b02-7690-4969-a89f-ac1af1c80435
│      Assigned to: Podcast Script Snippet Writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Podcast Script Snippet Writer
│           Status: ✅ Completed
└── 📋 Task: 2d38dbbd-6633-4b99-a50d-8be98c1390df
       Assigned to: Episode Plan Consolidator
       Status: ✅ Completed
    └── 🤖 Agent: Episode Plan Consolidator
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 2d38dbbd-6633-4b99-a50d-8be98c1390df                                                                     │
│  Agent: Episode Plan Consolidator                                                                               │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: f4a0c33a-87a2-4047-98c5-8840131bd647                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯



*****************************************
   PODCAST PLANNING CREW FINISHED WORK!   
*****************************************


In [11]:
# @title 8. Display final episode plan

print("\n\n#####################################")
print("## Generated podcast episode plan:")
print("#####################################\n")

if 'result' in locals():
    # Print the final output from the 'plan_compiler' agent
    print(result)
else:
    print("!!! The 'result' variable is not defined. Ensure Cell 7 executed correctly.")



#####################################
## Generated podcast episode plan:
#####################################

# Podcast Episode Plan: "The Future of Remote Work and Its Impact on Company Culture" Featuring Dr. Aria Patel

## 1. Episode Overview
- **Topic**: The Future of Remote Work and Its Impact on Company Culture
- **Guest**: Dr. Aria Patel, an expert in educational psychology from Karnataka, India, specializing in the psychological aspects of technology adaptation in educational and corporate settings.

## 2. Research Summary Highlights
- **Flexibility and Innovation**: Remote work encourages flexibility which can lead to innovations in work processes and employee engagement.
- **Communication Challenges**: The shift to digital communication tools affects the fluidity of team interactions.
- **Diversity and Inclusion**: Virtual environments could limit diverse interactions if not managed effectively.
- **Work-Life Balance**: While remote work increases flexibility, it may blur 